In [1]:
import pandas as pd
import numpy as np
import random as rd
from sklearn.preprocessing import StandardScaler as ss

In [2]:
def sig(x):
    x_64=x.astype(float)
    return 1/(1+np.exp(-x_64))

In [3]:
#create dataset from titanic dataset
data=pd.read_csv("train.csv")[["Survived","Pclass","Sex","Age","Fare","SibSp"]].dropna()
X_train=data[["Pclass","Sex","Age","Fare","SibSp"]]
Y_train=data[["Survived"]]

In [4]:
X_train.loc[X_train.Sex=="male","Sex"]=1
X_train.loc[X_train.Sex=="female","Sex"]=0

In [5]:
#normalize our data
s=ss()
X_train=s.fit_transform(X_train)

In [6]:
Y_train=np.array(Y_train)

In [60]:

X=X_train[0:2]
Y=Y_train[0:2]

In [61]:
print(X)
print(Y)

[[ 0.91123237  0.75905134 -0.53037664 -0.51897787  0.52457013]
 [-1.47636364 -1.31743394  0.57183099  0.69189675  0.52457013]]
[[0]
 [1]]


In [62]:
def train(X,Y,*arch,lr=1,l=0.4):
    weights=[]
    start_cap_deltas=[]
    for i in range(len(arch)-1):
        weights.append(np.array([[rd.uniform(-0.5,0.5) for k in range(arch[i])] for j in range(arch[i+1])]))
        start_cap_deltas.append(np.array([[0 for k in range(arch[i])] for j in range(arch[i+1])]))
        #creating weight matrices and weights updates accumulators according to ANN architecture
    print(start_cap_deltas)
    print(weights)
    k=0
    delta_cost=1
    cost_values=[] 
    cap_deltas=[]
    while lr>=0.00001:  
        delta_cost=1
        while(delta_cost>=0.00001): 
            cap_deltas.clear()
            for i in range(len(arch)-1):
                cap_deltas.append(np.array([[0 for k in range(arch[i])] for j in range(arch[i+1])]))
            print("capitals deltas when iteration started is {0}".format(start_cap_deltas))
            k+=1
            sum_er=0
            
            if k>2:
                del cost_values[0]
            for t in range(len(X)):
                #forward propagation
                a=[]
                z=[]
                for i in range(len(arch)):
                    if i==0:
                        a.append(X[t].reshape(1,arch[0]))
                    else:
                        z.append(np.matmul(a[i-1],weights[i-1].transpose()))
                        a.append(sig(z[i-1]))

                #backpropagation
                E_total=-Y[t]*np.log(a[len(arch)-1])-(1-Y[t])*np.log(1-a[len(arch)-1])
                #print("For obj number {0} E_total is {1}, out is {2}".format(t,E_total,a[len(arch)-1]))
                sum_er+=E_total
                #print("E_total for {0} object is {1}".format(t,E_total))
                deltas=[]
                
                for i in range(len(arch)-1,0,-1): #calculating delta terms for each layer
                    if i==len(arch)-1:
                        deltas.append(a[i]-Y[t])
                    else:
                        deltas=[np.array(np.matmul(deltas[0],weights[i])*a[i]*(1-a[i]))]+deltas
                #deltas[0] corresponds to second delta term dletas[1] corresponds to third delta term etc, if numerating 
                #of layers started from 1
                
                d_weights=[]
                for i in range(len(weights)):#calculating d_weight for each weight
                    d_weights.append(np.matmul(deltas[i].transpose(),a[i]))
                    cap_deltas[i]=cap_deltas[i]+d_weights[i] #accumulate weights updates
                    #print("for obj {0} cap_deltas is {1}, d_weights is {2}".format(t,cap_deltas[0],d_weights[0]))
                if t==0:
                    print("it number {0},X[0] is {1}, \n thetas are {2} \n,z[0] is {3}, \n h[0] is {4}, \n delta is {5}, \n error is {6}".
                                 format(k,X[0],weights,z[0],a[len(arch)-1].reshape(1,),deltas,Y[t]-a[len(arch)-1].reshape(1,)))
                #if t==7:
                    #print("d_weights for obj {0} is {1}  delta is {2}, error is {3}".
                          #format(t,d_weights, deltas,Y[t]-a[len(arch)-1].reshape(1,)))
                    #print("it number {0},weights are {1},d_weights are {2}".format(k,weights[1],d_weights[1]))
                #if t==0:
                
            print("cap_deltas[0] are {0}".format(cap_deltas[0])) 
            print()
            for i in range(len(weights)):#change weights according to acuumulated values
                weights[i]=weights[i]-lr/len(X)*cap_deltas[i]
                #if i==1:
                    #print("mean cap_deltas are {0}".format(cap_deltas[1]/len(X)))
            cost=1/len(X)*sum_er
            cost_values.append(*cost)
            if k>=2:
                delta_cost=cost_values[0]-cost_values[1]
            
            print("it number {0}, cost is {1}".format(k,cost))
        else:
            lr/=2
            #print("we stopped cause cost function didn't change, lr ={0}".format(lr))
            #print("delta_cost=%f" % delta_cost)
        """
        массив a=выходы с нейронов, нулевой элемент- вектор выходов с первого слоя
        массив z=входы с нейронов, нулевой эл-т- вектор входов во второй нейрон
        массив weights=матрицы весов, нулевой эл-т- матрица весов с первого слоя на второй
        массив deltas=массив ошибок на входе каждого из слоев, кроме первого. Нулевой элемент-ошибка на входе нейронов второго слоя
        массив d_weights=массив необходимых изменений матриц весов после запуска нейронной сети для одного объекта. Нулевой эл-т-
        необходимое изменение матрицы весов с первого слоя на второй
        массив cap_deltas=массив необходимых изменений матриц весов после одного пробега по всем объектам матрицы входных значений.
        Нулевой эл-т-суммарное изменение матрицы весов с первого слоя на второй

        """
            
    return weights,arch

In [54]:
[[ 0.30581925,  0.19813939, -0.15974948, -0.3445205 ,  0.45721307]]

[[0.30581925, 0.19813939, -0.15974948, -0.3445205, 0.45721307]]

In [66]:
weights,arch=train(X,Y,5,1,1)

[array([[0, 0, 0, 0, 0]]), array([[0]])]
[array([[-0.44757316,  0.05527317, -0.10362545,  0.09137225, -0.1102073 ]]), array([[-0.08005341]])]
capitals deltas when iteration started is [array([[0, 0, 0, 0, 0]]), array([[0]])]
it number 1,X[0] is [ 0.91123237  0.75905134 -0.53037664 -0.51897787  0.52457013], 
 thetas are [array([[-0.44757316,  0.05527317, -0.10362545,  0.09137225, -0.1102073 ]]), array([[-0.08005341]])] 
,z[0] is [[-0.4161591]], 
 h[0] is [0.49204664], 
 delta is [array([[-0.00943315]]), array([[0.49204664]])], 
 error is [-0.49204664]
cap_deltas[0] are [[-2.27111416e-02 -1.97560860e-02  1.04703348e-02  1.15107442e-02
   6.70117731e-05]]

it number 1, cost is [[0.69803285]]
capitals deltas when iteration started is [array([[0, 0, 0, 0, 0]]), array([[0]])]
it number 2,X[0] is [ 0.91123237  0.75905134 -0.53037664 -0.51897787  0.52457013], 
 thetas are [array([[-0.43621759,  0.06515121, -0.10886061,  0.08561688, -0.11024081]]), array([[-0.01624504]])] 
,z[0] is [[-0.3925676

it number 100,X[0] is [ 0.91123237  0.75905134 -0.53037664 -0.51897787  0.52457013], 
 thetas are [array([[-2.38394323, -1.58380074,  0.9378411 ,  1.14985926, -0.81572641]]), array([[3.54867188]])] 
,z[0] is [[-4.89657855]], 
 h[0] is [0.50657947], 
 delta is [array([[0.01323398]]), array([[0.50657947]])], 
 error is [-0.50657947]
cap_deltas[0] are [[ 0.01227771  0.01024023 -0.00710361 -0.00697053  0.00686452]]

it number 100, cost is [[0.36744911]]
capitals deltas when iteration started is [array([[0, 0, 0, 0, 0]]), array([[0]])]
it number 101,X[0] is [ 0.91123237  0.75905134 -0.53037664 -0.51897787  0.52457013], 
 thetas are [array([[-2.39008208, -1.58892086,  0.9413929 ,  1.15334453, -0.81915866]]), array([[3.56082359]])] 
,z[0] is [[-4.91155194]], 
 h[0] is [0.5065046], 
 delta is [array([[0.01308289]]), array([[0.5065046]])], 
 error is [-0.5065046]
cap_deltas[0] are [[ 0.01213424  0.01012038 -0.00702124 -0.00688941  0.00678732]]

it number 101, cost is [[0.36720242]]
capitals del

capitals deltas when iteration started is [array([[0, 0, 0, 0, 0]]), array([[0]])]
it number 210,X[0] is [ 0.91123237  0.75905134 -0.53037664 -0.51897787  0.52457013], 
 thetas are [array([[-2.8160964 , -1.94406302,  1.18845055,  1.39550732, -1.06009447]]), array([[4.41458049]])] 
,z[0] is [[-5.95241958]], 
 h[0] is [0.5028615], 
 delta is [array([[0.0057409]]), array([[0.5028615]])], 
 error is [-0.5028615]
cap_deltas[0] are [[ 0.005263    0.00438593 -0.00305712 -0.00299426  0.00300024]]

it number 210, cost is [[0.35546755]]
capitals deltas when iteration started is [array([[0, 0, 0, 0, 0]]), array([[0]])]
it number 211,X[0] is [ 0.91123237  0.75905134 -0.53037664 -0.51897787  0.52457013], 
 thetas are [array([[-2.8187279 , -1.94625599,  1.18997911,  1.39700445, -1.06159459]]), array([[4.41991424]])] 
,z[0] is [[-5.95885666]], 
 h[0] is [0.50284663], 
 delta is [array([[0.00571098]]), array([[0.50284663]])], 
 error is [-0.50284663]
cap_deltas[0] are [[ 0.00523536  0.00436288 -0.0030

cap_deltas[0] are [[ 0.00332939  0.00277403 -0.00193567 -0.00189507  0.00190619]]

it number 319, cost is [[0.3521939]]
capitals deltas when iteration started is [array([[0, 0, 0, 0, 0]]), array([[0]])]
it number 320,X[0] is [ 0.91123237  0.75905134 -0.53037664 -0.51897787  0.52457013], 
 thetas are [array([[-3.04454302, -2.1344198 ,  1.32121281,  1.52550999, -1.19063139]]), array([[4.8801698]])] 
,z[0] is [[-6.51143638]], 
 h[0] is [0.5018107], 
 delta is [array([[0.00362914]]), array([[0.5018107]])], 
 error is [-0.5018107]
cap_deltas[0] are [[ 0.00331812  0.00276464 -0.00192912 -0.00188866  0.00189978]]

it number 320, cost is [[0.35217486]]
capitals deltas when iteration started is [array([[0, 0, 0, 0, 0]]), array([[0]])]
it number 321,X[0] is [ 0.91123237  0.75905134 -0.53037664 -0.51897787  0.52457013], 
 thetas are [array([[-3.04620208, -2.13580212,  1.32217738,  1.52645433, -1.19158129]]), array([[4.88356962]])] 
,z[0] is [[-6.51549737]], 
 h[0] is [0.50180463], 
 delta is [arr

it number 426,X[0] is [ 0.91123237  0.75905134 -0.53037664 -0.51897787  0.52457013], 
 thetas are [array([[-3.19497145, -2.25975069,  1.40868709,  1.61114185, -1.27683868]]), array([[5.18949032]])] 
,z[0] is [[-6.87970133]], 
 h[0] is [0.50133291], 
 delta is [array([[0.00267019]]), array([[0.50133291]])], 
 error is [-0.50133291]
cap_deltas[0] are [[ 0.0024387   0.00203176 -0.00141835 -0.00138837  0.00139873]]

it number 426, cost is [[0.35068984]]
capitals deltas when iteration started is [array([[0, 0, 0, 0, 0]]), array([[0]])]
it number 427,X[0] is [ 0.91123237  0.75905134 -0.53037664 -0.51897787  0.52457013], 
 thetas are [array([[-3.19619081, -2.26076657,  1.40939627,  1.61183603, -1.27753804]]), array([[5.19200627]])] 
,z[0] is [[-6.88268681]], 
 h[0] is [0.50132959], 
 delta is [array([[0.00266352]]), array([[0.50132959]])], 
 error is [-0.50132959]
cap_deltas[0] are [[ 0.00243259  0.00202666 -0.0014148  -0.00138489  0.00139524]]

it number 427, cost is [[0.35067953]]
capitals 

In [40]:
def predict(X,weights,arch):
    print(arch)
    predictions=[]
    for t in range(len(X)):
    #forward propagation
        a=[]
        z=[]
        for i in range(len(arch)):
            if i==0:
                a.append(X[t].reshape(1,arch[0]))
            else:
                z.append(np.matmul(a[i-1],weights[i-1].transpose()))
                a.append(sig(z[i-1]))
        predictions.append(a[len(arch)-1])
    return predictions       

In [67]:
pred=np.array(predict(X,weights,arch))
pred=pred.reshape(len(pred),)

(5, 1, 1)


In [68]:
print(pred)

[0.50130038 0.99458804]


In [29]:
pred_rounded=list(map(round,pred))

In [30]:
print(Y)

[[0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]


In [31]:
k=0
for i in zip(pred_rounded,Y):
    if i[0]==i[1]:
        k+=1  
print("accuracy=",k/len(Y))

accuracy= 0.7913165266106442


[[5 6 7]]
